In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [31]:
df = pd.read_csv("../data/train_data.csv")
# removing duplicates
df = df.drop_duplicates()
# no. of nan values
df.isnull().sum().sum()

np.int64(0)

In [32]:
df.head()

,age,gender,income_bracket,loyalty_program,membership_years,churned,marital_status,number_of_children,education_level,occupation,...,distance_to_store,holiday_season,season,weekend,customer_support_calls,email_subscriptions,app_usage,website_visits,social_media_engagement,days_since_last_purchase
0,41,Female,Medium,No,3,No,Married,0,Bachelor's,Employed,...,72.83,No,Summer,No,18,No,Medium,2,Medium,6
1,40,Female,High,Yes,4,Yes,Single,4,PhD,Employed,...,44.60,Yes,Fall,No,16,Yes,Low,17,Medium,137
2,48,Male,High,No,4,Yes,Married,3,Bachelor's,Unemployed,...,46.15,Yes,Winter,Yes,11,Yes,High,68,High,275
3,19,Male,Low,No,9,Yes,Divorced,3,Bachelor's,Retired,...,0.62,No,Fall,No,15,Yes,Medium,61,Low,182
4,53,Female,Low,Yes,8,No,Divorced,2,Master's,Employed,...,38.09,Yes,Fall,Yes,0,Yes,High,19,High,330


In [33]:
# number of unique values for each column
nunique_counts = df.nunique()
cols_less_than = nunique_counts[nunique_counts == 2]
cols_less_than.count()

np.int64(6)

In [34]:
# perform a correlation test between numerical columns and the target variable 'Price'
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('avg_purchase_value')
for i in numerical_cols:
    correlation = df[i].corr(df['avg_purchase_value'])
    # if abs(correlation) > 0.05:
    print(f"Correlation between {i} and avg_purchase_value: {correlation}")

Correlation between age and avg_purchase_value: 0.00022571172476673076
Correlation between membership_years and avg_purchase_value: -0.0017891850521989272
Correlation between number_of_children and avg_purchase_value: -0.0013622426741891778
Correlation between transaction_id and avg_purchase_value: 0.0017257036005454247
Correlation between product_id and avg_purchase_value: -1.8832782850825444e-05
Correlation between quantity and avg_purchase_value: 0.0022717571246670095
Correlation between unit_price and avg_purchase_value: -0.0008484805528193874
Correlation between discount_applied and avg_purchase_value: -0.0013247792335617083
Correlation between transaction_hour and avg_purchase_value: -0.0010808926027579522
Correlation between week_of_year and avg_purchase_value: 0.0008584929001343206
Correlation between month_of_year and avg_purchase_value: -0.0011885867745611305
Correlation between avg_discount_used and avg_purchase_value: -0.0024858504275598737
Correlation between online_purcha

In [35]:
# do chi squared test for correlation for categorical data
# from scipy.stats import chi2_contingency
# categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
# for q in [3, 4, 5]:
#     df[f'avg_purchase_value_bin_{q}'] = pd.qcut(df['avg_purchase_value'], q=q, duplicates='drop')
#     print(f"\n=== Using {q} quantile bins ===")
#     for col in categorical_cols:
#         contingency_table = pd.crosstab(df[col], df[f'avg_purchase_value_bin_{q}'])
#         if (contingency_table.values < 5).sum() > 0:
#             print(f"⚠️ Warning: some cells < 5 counts for {col}")
#         chi2, p, dof, expected = chi2_contingency(contingency_table)
#         print(f"{col}: p-value = {p:.4f}")


In [36]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
max_categories = 30  # tweak depending on dataset
filtered_categorical_cols = [col for col in categorical_cols if df[col].nunique() <= max_categories]
sample_df = df.sample(n=10000, random_state=42)


import statsmodels.api as sm
from statsmodels.formula.api import ols

alpha = 0.05
results = []

for col in filtered_categorical_cols:
    # Drop missing values only for the relevant columns
    sub_df = sample_df[[col, "avg_purchase_value"]].dropna()
    
    if sub_df[col].nunique() < 2:
        continue  # skip if column has only one unique category
    
    try:
        model = ols(f"avg_purchase_value ~ C({col})", data=sub_df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        p_value = anova_table["PR(>F)"][0]
        if p_value < alpha:
            results.append((col, p_value))
    except Exception as e:
        print(f"Skipping {col} due to error: {e}")

# Sort by p-value
results = sorted(results, key=lambda x: x[1])
# for col, p_value in results:
#     print(f"{col}: p-value = {p_value:.4f}")
results

/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_94034/2571292036.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_table["PR(>F)"][0]
/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_94034/2571292036.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_table["PR(>F)"][0]
/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_94034/2571292036.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, u

[('holiday_season', np.float64(0.0017703432772197018)),
 ('occupation', np.float64(0.018936171069983205)),
 ('promotion_type', np.float64(0.0397215853569443)),
 ('payment_method', np.float64(0.047947021353577354))]

In [37]:
date_cols = []

# collect date columns
for i in df.columns:
    if 'date' in i.lower():
        date_cols.append(i)

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
    df[col] = df[col].astype('int64') / 1e9  # convert to float seconds

In [38]:
numeric_cols = df.select_dtypes(include=['number']).columns

#Outlier remover
def whisker(col):
    Q1,Q3 = np.percentile(col,[25,75])
    iqr = Q3 - Q1
    lw = Q1 - (1.5 * iqr)
    uw = Q3 + (1.5 * iqr)
    return lw, uw

In [39]:
for i in numeric_cols:
    lw,uw = whisker(df[i])
    df[i] = np.where(df[i]<lw, lw, df[i])
    df[i] = np.where(df[i]>uw, uw, df[i])
    # sns.boxplot(data=df, x=i)
    # plt.show()

In [40]:
# Data normalization

# to_normalize = [
#     "age",
#     "membership_years",
#     "number_of_children",
#     "transaction_id",
#     "product_id",
#     "quantity",
#     "unit_price",
#     "discount_applied",
#     "transaction_hour",
#     "week_of_year",
#     "month_of_year",
#     "avg_purchase_value",
#     "avg_discount_used",
#     "online_purchases",
#     "in_store_purchases",
#     "avg_items_per_transaction",
#     "avg_transaction_value",
#     "total_returned_items",
#     "total_returned_value",
#     "total_sales",
#     "total_transactions",
#     "total_items_purchased",
#     "total_discounts_received",
#     "avg_spent_per_category",
#     "max_single_purchase_value",
#     "min_single_purchase_value",
#     "product_rating",
#     "product_review_count",
#     "product_stock",
#     "product_return_rate",
#     "product_weight",
#     "product_shelf_life",
#     "promotion_id",
#     "customer_zip_code",
#     "store_zip_code",
#     "distance_to_store",
#     "customer_support_calls",
#     "website_visits",
#     "days_since_last_purchase"
# ]

# to normalize all numerical columns
to_normalize = numeric_cols.tolist()

# to_normalize.append("FlashStorage")
for col in to_normalize:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col] = (df[col] - min_val) / (max_val - min_val)

# for i in to_normalize:
#     sns.scatterplot(data=df, x=i, y='Price')
#     plt.show())

# df.drop(columns=["FlashStorage"], inplace=True)
# df["education_level_High School"]

In [41]:
# df = df.dropna(axis=1)
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

binary_cols = []
multi_cols = []

# Separate columns based on number of unique values
for col in categorical_cols:
    unique_vals = df[col].dropna().unique()
    if len(unique_vals) == 2:
        binary_cols.append(col)
    elif len(unique_vals) > 2:
        multi_cols.append(col)

# Encode binary columns as 0/1
for col in binary_cols:
    # Map one category to 0 and the other to 1
    unique_vals = sorted(df[col].dropna().unique())  # sort for reproducibility
    df[col] = df[col].map({unique_vals[0]: 0, unique_vals[1]: 1})

# One-hot encode multi-category columns
df = pd.get_dummies(df, columns=multi_cols, drop_first=False, dtype=float)

df.head()

,age,loyalty_program,membership_years,churned,number_of_children,transaction_id,transaction_date,product_id,quantity,unit_price,...,season_Fall,season_Spring,season_Summer,season_Winter,app_usage_High,app_usage_Low,app_usage_Medium,social_media_engagement_High,social_media_engagement_Low,social_media_engagement_Medium
0,0.377049,0,0.333333,0,0.00,0.237854,0.102799,0.099620,0.750,0.960581,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.360656,1,0.444444,1,1.00,0.500661,0.118543,0.286857,0.500,0.218739,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.491803,0,0.444444,1,0.75,0.432551,0.189110,0.617223,0.875,0.759710,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.016393,0,1.000000,1,0.75,0.771343,0.906700,0.600420,1.000,0.862012,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.573770,1,0.888889,0,0.50,0.186218,0.508477,0.356471,0.250,0.609419,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [42]:
# output_path = "/Users/avi/Desktop/university/sem_5/Machine_Learning/Avi_Dhall_A1/retail_task/data/cleaned_data.csv"
# df.to_csv(output_path, index=False)